In [ ]:
!python setup.py install &> /dev/null

In [6]:
import argparse, pytorch_lightning as pl
from pytorch_lightning import Trainer
from src.trainers import ContrastiveLearning
from src.utils import yaml_config_hook
from src.loaders import get_dataset
from torch.utils.data import DataLoader


# create args parser and link to Lightning trainer:
parser = argparse.ArgumentParser(description="demo")
parser = Trainer.add_argparse_args(parser)

# extract args from config file and add to parser:
config = yaml_config_hook("config/config_ptbxl.yaml")
for key, value in config.items():
    parser.add_argument(f"--{key}", default=value, type=type(value))
args = parser.parse_args()

# set random seed if selected:
if args.seed:
    pl.seed_everything(args.seed, workers=True)

test_dataset = get_dataset(
    dataset=args.dataset,
    dataset_dir=args.dataset_dir,
    sr=args.sr,
    split="test",
    gtruth=args.gtruth,
)

test_loader = DataLoader(
    test_dataset,
    batch_size=args.batch_size,
    shuffle=False,
    num_workers=args.workers,
    drop_last=False,
)


Global seed set to 42


Loading from cache...
Loaded 2196 ECG samples in total.


In [7]:
from src.models import ResNet1D

encoder = ResNet1D(
    in_channels=args.in_channels,
    base_filters=args.base_filters,
    kernel_size=args.kernel_size,
    stride=args.stride,
    groups=args.groups,
    n_block=args.n_block,
    n_classes=args.n_classes,
)

ckpt_path = "ckpt/BIG_epoch=99-step=97300.ckpt"
model = ContrastiveLearning.load_from_checkpoint(ckpt_path, encoder=encoder)

In [4]:
from src.utils import evaluate
import torch

metrics, _ = evaluate(
    model.to(torch.device("cpu")),
    dataset=test_loader,
    dataset_name=args.dataset,
)

Evaluating model...


  0%|          | 0/9 [00:00<?, ?it/s]

data torch.Size([256, 1000])
data tensor([[ 0.4734,  0.8819,  1.1655,  ..., -1.0835, -0.8118, -0.0294],
        [ 0.0104,  0.2959, -1.7159,  ..., -0.4874, -0.0981,  0.1750],
        [-0.1190,  0.0362,  0.2389,  ..., -1.3782, -0.8564,  0.0424],
        ...,
        [-0.1013, -0.1572, -0.2031,  ..., -0.2356, -0.1528, -0.0526],
        [ 0.1494,  0.1372,  0.1251,  ..., -0.1884, -0.2308,  0.0739],
        [ 0.0145, -1.0351, -1.1175,  ...,  0.9049,  0.5642,  0.0998]],
       dtype=torch.float64)
label torch.Size([256, 5])
label tensor([[1, 0, 0, 0, 0],
        [1, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        ...,
        [0, 0, 1, 0, 0],
        [0, 0, 1, 1, 0],
        [0, 1, 0, 1, 0]])


  0%|          | 0/9 [00:48<?, ?it/s]


TypeError: iteration over a 0-d tensor

In [ ]:
from src.models import S4Model

encoder = S4Model(
    d_input=args.d_input,
    d_output=args.d_output,
    d_model=args.d_model,
    n_layers=args.n_layers,
    dropout=args.dropout,
    prenorm=True,
)

ckpt_path = "ckpt/s4_contr.ckpt"
model = ContrastiveLearning.load_from_checkpoint(ckpt_path, encoder=encoder)

/Users/andresalvarezolmo/.pyenv/versions/3.10.16/lib/python3.10/site-packages/lightning_fabric/utilities/cloud_io.py:48: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return